In [3]:
##################################################################################################
###############Building the dataset for testing of the teams classified World Cup 2018############
##################################################################################################

In [1]:
# Pandas and numpy for data manipulation
import pandas as pd
import numpy as np
np.random.seed(42)

# Matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

import matplotlib
matplotlib.rcParams['font.size'] = 10
matplotlib.rcParams['figure.figsize'] = (12, 12)

import seaborn as sns

from IPython.core.pylabtools import figsize

# Scipy helper functions
from scipy.stats import percentileofscore
from scipy import stats
from scipy.stats import poisson

# Stats Models
import statsmodels.api as sm
import statsmodels.formula.api as smf
import itertools
from IPython.display import display, HTML
import random

C:\Users\Hector Repilloza\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# Standard ML Models for comparison
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor


# Splitting data into training/testing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error

# Distributions
import scipy
from scipy.stats import pearsonr

from itertools import product

In [3]:
#Creating the evaluation dataset with the last statistics of the teams classified to the World Cup 2018
#Read the csv file WorldCup2018
WorldCup2018_Analytics_Sim = pd.read_excel('WorldCup2018_Analytics_Sim.xlsx')

In [4]:
#Convert Date from object to date format
WorldCup2018_Analytics_Sim['Date'] = pd.to_datetime(WorldCup2018_Analytics_Sim['Date'])

In [5]:
WorldCup2018_Analytics_Home = WorldCup2018_Analytics_Sim[['Date','Home_Team_Fifa_Code','Tournament_Weight','Friendly',
                                                          'Qualification','Final','FIFAWorldCup',
                                                          'Home_Conf_Weight','Home','Date_Weight',
                                                          'Home_Team_UEFA','last5games_w_per_home',
                                                          'last10games_w_per_home','last20games_w_per_home',
                                                          'last5games_l_per_home','last10games_l_per_home',
                                                          'last20games_l_per_home','last5games_d_per_home',
                                                          'last10games_d_per_home','last20games_d_per_home',
                                                          'last5games_gd_per_home','last10games_gd_per_home',
                                                          'last20games_gd_per_home','last5games_opp_cc_per_home',
                                                          'last10games_opp_cc_per_home','last20games_opp_cc_per_home']]

WorldCup2018_Analytics_Away =  WorldCup2018_Analytics_Sim[['Date','Away_Team_Fifa_Code','Tournament_Weight','Friendly',
                                                          'Qualification','Final','FIFAWorldCup',
                                                          'Away_Conf_Weight','Away','Date_Weight',
                                                          'Away_Team_UEFA','last5games_w_per_away',
                                                           'last10games_w_per_away','last20games_w_per_away',
                                                           'last5games_l_per_away','last10games_l_per_away',
                                                          'last20games_l_per_away','last5games_d_per_away',
                                                          'last10games_d_per_away','last20games_d_per_away',
                                                          'last5games_gd_per_away','last10games_gd_per_away',
                                                          'last20games_gd_per_away','last5games_opp_cc_per_away',
                                                           'last10games_opp_cc_per_away','last20games_opp_cc_per_away']]

In [6]:
#Filter the Dataset with the qualify teams for World Cup 2018 
filter_list_teams = ['RUS', 'KSA', 'EGY', 'URU',
                     'POR', 'ESP', 'MAR', 'IRN',
                     'FRA', 'AUS', 'PER', 'DEN',
                     'ARG', 'ISL', 'CRO', 'NGA',
                     'BRA', 'SUI', 'CRC', 'SRB',
                     'GER', 'SWE', 'MEX', 'KOR',
                     'BEL', 'PAN', 'TUN', 'ENG',
                     'POL', 'SEN', 'COL', 'JPN',]

WorldCup2018_Analytics_Home= WorldCup2018_Analytics_Home[WorldCup2018_Analytics_Home.Home_Team_Fifa_Code.isin(filter_list_teams)]
WorldCup2018_Analytics_Away= WorldCup2018_Analytics_Away[WorldCup2018_Analytics_Away.Away_Team_Fifa_Code.isin(filter_list_teams)]

In [7]:
#Home Last Results of the 32 Teams in the World Cup 2018
WorldCup2018_Analytics_Home_32 = WorldCup2018_Analytics_Home.sort_values(by='Date')
WorldCup2018_Analytics_Home_32 = WorldCup2018_Analytics_Home.sort_index().drop_duplicates('Home_Team_Fifa_Code', 
                                                                                          keep='last').reset_index()
WorldCup2018_Analytics_Home_32 = WorldCup2018_Analytics_Home_32.drop('index', 1)


#Away Last Results of the 32 Teams in the World Cup 2018
WorldCup2018_Analytics_Away_32 = WorldCup2018_Analytics_Away.sort_values(by='Date')
WorldCup2018_Analytics_Away_32 = WorldCup2018_Analytics_Away.sort_index().drop_duplicates('Away_Team_Fifa_Code',
                                                                                          keep='last').reset_index()
WorldCup2018_Analytics_Away_32 = WorldCup2018_Analytics_Away_32.drop('index', 1)

In [8]:
#Set Tournament_Weight into 'FIFA World Cup' = 4 for all teams
WorldCup2018_Analytics_Home_32.Tournament_Weight=4

#Set Friendly into False for all teams
WorldCup2018_Analytics_Home_32['Friendly'] = False

#Set Qualification into False for all teams
WorldCup2018_Analytics_Home_32['Qualification'] = False

#Set Final into False for all teams
WorldCup2018_Analytics_Home_32['Final'] = False

#Set FIFAWorldCup into False for all teams
WorldCup2018_Analytics_Home_32['FIFAWorldCup'] = True

#Set Home into RUS = True else False
WorldCup2018_Analytics_Home_32 = WorldCup2018_Analytics_Home_32.drop('Home', 1)
WorldCup2018_Analytics_Home_32['Home'] = np.where(WorldCup2018_Analytics_Home_32.Home_Team_Fifa_Code == 'RUS', True, False)

#Set Date_Weight into 'Date_Weight' = 4 for all teams
WorldCup2018_Analytics_Home_32.Date_Weight = 1

#Rearrange the columns and drop Date
WorldCup2018_Analytics_Home_32 = WorldCup2018_Analytics_Home_32[['Home_Team_Fifa_Code','Tournament_Weight','Friendly',
                                                          'Qualification','Final','FIFAWorldCup',
                                                          'Home_Conf_Weight','Home','Date_Weight',
                                                          'Home_Team_UEFA','last5games_w_per_home',
                                                          'last10games_w_per_home','last20games_w_per_home',
                                                          'last5games_l_per_home','last10games_l_per_home',
                                                          'last20games_l_per_home','last5games_d_per_home',
                                                          'last10games_d_per_home','last20games_d_per_home',
                                                          'last5games_gd_per_home','last10games_gd_per_home',
                                                          'last20games_gd_per_home','last5games_opp_cc_per_home',
                                                          'last10games_opp_cc_per_home','last20games_opp_cc_per_home']]

In [9]:
#Set Tournament_Weight into 'FIFA World Cup' = 4 for all teams
WorldCup2018_Analytics_Away_32.Tournament_Weight=4

#Set Friendly into False for all teams
WorldCup2018_Analytics_Away_32['Friendly'] = False

#Set Qualification into False for all teams
WorldCup2018_Analytics_Away_32['Qualification'] = False

#Set Final into False for all teams
WorldCup2018_Analytics_Away_32['Final'] = False

#Set FIFAWorldCup into False for all teams
WorldCup2018_Analytics_Away_32['FIFAWorldCup'] = True

#Set Home into RUS = True else False
WorldCup2018_Analytics_Away_32 = WorldCup2018_Analytics_Away_32.drop('Away', 1)
WorldCup2018_Analytics_Away_32['Away'] = np.where(WorldCup2018_Analytics_Away_32.Away_Team_Fifa_Code == 'RUS', False, True)

#Set Date_Weight into 'Date_Weight' = 4 for all teams
WorldCup2018_Analytics_Away_32.Date_Weight = 1

#Rearrange the columns and drop Date
WorldCup2018_Analytics_Away_32 =  WorldCup2018_Analytics_Away_32[['Away_Team_Fifa_Code','Tournament_Weight','Friendly',
                                                          'Qualification','Final','FIFAWorldCup',
                                                          'Away_Conf_Weight','Away','Date_Weight',
                                                          'Away_Team_UEFA','last5games_w_per_away',
                                                           'last10games_w_per_away','last20games_w_per_away',
                                                           'last5games_l_per_away','last10games_l_per_away',
                                                          'last20games_l_per_away','last5games_d_per_away',
                                                          'last10games_d_per_away','last20games_d_per_away',
                                                          'last5games_gd_per_away','last10games_gd_per_away',
                                                          'last20games_gd_per_away','last5games_opp_cc_per_away',
                                                           'last10games_opp_cc_per_away','last20games_opp_cc_per_away']]

In [10]:
WorldCup2018_Analytics_Away_32.head()

,Away_Team_Fifa_Code,Tournament_Weight,Friendly,Qualification,Final,FIFAWorldCup,Away_Conf_Weight,Away,Date_Weight,Away_Team_UEFA,...,last20games_l_per_away,last5games_d_per_away,last10games_d_per_away,last20games_d_per_away,last5games_gd_per_away,last10games_gd_per_away,last20games_gd_per_away,last5games_opp_cc_per_away,last10games_opp_cc_per_away,last20games_opp_cc_per_away
0,RUS,4,False,False,False,True,0.99,False,1,1,...,0.30,0.4,0.3,0.30,0.8,0.0,-0.30,0.99,0.99,0.99
1,POL,4,False,False,False,True,0.99,True,1,1,...,0.40,0.4,0.5,0.35,0.0,0.1,-0.45,0.99,0.99,0.99
2,TUN,4,False,False,False,True,0.85,True,1,0,...,0.35,0.2,0.2,0.35,0.0,-0.8,-0.30,0.85,0.85,0.85
3,POR,4,False,False,False,True,0.99,True,1,1,...,0.65,0.0,0.2,0.15,-3.0,-1.5,-0.95,0.99,0.99,0.99
4,BEL,4,False,False,False,True,0.99,True,1,1,...,0.65,0.4,0.2,0.15,-1.8,-1.6,-1.10,0.99,0.99,0.99


In [11]:
#Read the csv file Qualified_Teams_WC2018
Qualified_Teams_WC2018_Home = pd.read_excel('Qualified_Teams_WC2018_Home.xlsx')
Qualified_Teams_WC2018_Away = pd.read_excel('Qualified_Teams_WC2018_Away.xlsx')
Qualified_Teams_WC2018 = pd.read_excel('Qualified_Teams_WC2018.xlsx')

In [12]:
#All possible combinations of matches between the Qualified_Teams_WC2018
from itertools import product

Qualified_Teams_WC2018_1 = Qualified_Teams_WC2018['Home_Team_Fifa_Code']
Qualified_Teams_WC2018_2 = Qualified_Teams_WC2018['Away_Team_Fifa_Code']

WorldCup_Matches = pd.DataFrame(list(product(Qualified_Teams_WC2018_1, Qualified_Teams_WC2018_2)), 
             columns=['Home_Team_Fifa_Code', 'Away_Team_Fifa_Code'])

#Remove rows where Home_Team_Fifa_Code == Away_Team_Fifa_Code
WorldCup_Matches = WorldCup_Matches[WorldCup_Matches.Home_Team_Fifa_Code != WorldCup_Matches.Away_Team_Fifa_Code ]


#Adding Draws
#Draw Home
WorldCup_Matches = WorldCup_Matches.merge(Qualified_Teams_WC2018_Home, 
                        left_on=['Home_Team_Fifa_Code'], 
                        right_on=['Team_Fifa_Code'])

WorldCup_Matches = WorldCup_Matches.drop('Team_Fifa_Code', 1)

#Draw Away
WorldCup_Matches = WorldCup_Matches.merge(Qualified_Teams_WC2018_Away, 
                        left_on=['Away_Team_Fifa_Code'], 
                        right_on=['Team_Fifa_Code'])

WorldCup_Matches = WorldCup_Matches.drop('Team_Fifa_Code', 1)


In [13]:
WorldCup_Matches.head()

,Home_Team_Fifa_Code,Away_Team_Fifa_Code,Draw_Home,Draw_Away
0,RUS,KSA,A1,A2
1,EGY,KSA,A3,A2
2,URU,KSA,A4,A2
3,POR,KSA,B1,A2
4,ESP,KSA,B2,A2


In [14]:
WorldCup_Matches.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 992 entries, 0 to 991
Data columns (total 4 columns):
Home_Team_Fifa_Code    992 non-null object
Away_Team_Fifa_Code    992 non-null object
Draw_Home              992 non-null object
Draw_Away              992 non-null object
dtypes: object(4)
memory usage: 38.8+ KB


In [15]:
#Create final test set and merge WorldCup_Matches, WorldCup2018_Analytics_Home_32 and WorldCup2018_Analytics_Away_32
#Home
WorldCup_Matches_Analytics = WorldCup_Matches.merge(WorldCup2018_Analytics_Home_32, 
                        left_on=['Home_Team_Fifa_Code'], 
                        right_on=['Home_Team_Fifa_Code'])

#Away
WorldCup2018_Analytics_Away_32 =  WorldCup2018_Analytics_Away_32[['Away_Team_Fifa_Code',
                                                          'Away_Conf_Weight','Away',
                                                          'Away_Team_UEFA','last5games_w_per_away',
                                                           'last10games_w_per_away','last20games_w_per_away',
                                                           'last5games_l_per_away','last10games_l_per_away',
                                                          'last20games_l_per_away','last5games_d_per_away',
                                                          'last10games_d_per_away','last20games_d_per_away',
                                                          'last5games_gd_per_away','last10games_gd_per_away',
                                                          'last20games_gd_per_away','last5games_opp_cc_per_away',
                                                           'last10games_opp_cc_per_away','last20games_opp_cc_per_away']]

WorldCup_Matches_Analytics = WorldCup_Matches_Analytics.merge(WorldCup2018_Analytics_Away_32, 
                        left_on=['Away_Team_Fifa_Code'], 
                        right_on=['Away_Team_Fifa_Code'])


In [16]:
WorldCup_Matches_Analytics.head()

,Home_Team_Fifa_Code,Away_Team_Fifa_Code,Draw_Home,Draw_Away,Tournament_Weight,Friendly,Qualification,Final,FIFAWorldCup,Home_Conf_Weight,...,last20games_l_per_away,last5games_d_per_away,last10games_d_per_away,last20games_d_per_away,last5games_gd_per_away,last10games_gd_per_away,last20games_gd_per_away,last5games_opp_cc_per_away,last10games_opp_cc_per_away,last20games_opp_cc_per_away
0,RUS,KSA,A1,A2,4,False,False,False,True,0.99,...,0.3,0.4,0.2,0.2,1.2,0.9,0.25,0.85,0.85,0.85
1,EGY,KSA,A3,A2,4,False,False,False,True,0.85,...,0.3,0.4,0.2,0.2,1.2,0.9,0.25,0.85,0.85,0.85
2,URU,KSA,A4,A2,4,False,False,False,True,1.00,...,0.3,0.4,0.2,0.2,1.2,0.9,0.25,0.85,0.85,0.85
3,POR,KSA,B1,A2,4,False,False,False,True,0.99,...,0.3,0.4,0.2,0.2,1.2,0.9,0.25,0.85,0.85,0.85
4,ESP,KSA,B2,A2,4,False,False,False,True,0.99,...,0.3,0.4,0.2,0.2,1.2,0.9,0.25,0.85,0.85,0.85


In [20]:
#Filter WorldCup_Matches_Analytics
#WorldCup_Matches_Analytics = WorldCup_Matches_Analytics[(WorldCup_Matches_Analytics['Home_Team_Fifa_Code']=='BRA') & 
#                                                        (WorldCup_Matches_Analytics['Away_Team_Fifa_Code']=='ARG')]
#WorldCup_Matches_Analytics.head()

In [17]:
#Create the same format to predict on the test set
WorldCup_Matches_Test = WorldCup_Matches_Analytics[['Tournament_Weight','Friendly',
                                                          'Qualification','Final','FIFAWorldCup',
                                                          'Home_Conf_Weight', 'Away_Conf_Weight',
                                                          'Home','Away','Date_Weight',
                                                          'Home_Team_UEFA','Away_Team_UEFA',
                                                          'last5games_w_per_home',
                                                          'last10games_w_per_home','last20games_w_per_home',
                                                          'last5games_l_per_home','last10games_l_per_home',
                                                          'last20games_l_per_home','last5games_d_per_home',
                                                          'last10games_d_per_home','last20games_d_per_home',
                                                          'last5games_gd_per_home','last10games_gd_per_home',
                                                          'last20games_gd_per_home','last5games_opp_cc_per_home',
                                                          'last10games_opp_cc_per_home','last20games_opp_cc_per_home',                  
                                                           'last5games_w_per_away',
                                                           'last10games_w_per_away','last20games_w_per_away',
                                                           'last5games_l_per_away','last10games_l_per_away',
                                                          'last20games_l_per_away','last5games_d_per_away',
                                                          'last10games_d_per_away','last20games_d_per_away',
                                                          'last5games_gd_per_away','last10games_gd_per_away',
                                                          'last20games_gd_per_away','last5games_opp_cc_per_away',
                                                           'last10games_opp_cc_per_away','last20games_opp_cc_per_away']]
WorldCup_Matches_Test.head()

,Tournament_Weight,Friendly,Qualification,Final,FIFAWorldCup,Home_Conf_Weight,Away_Conf_Weight,Home,Away,Date_Weight,...,last20games_l_per_away,last5games_d_per_away,last10games_d_per_away,last20games_d_per_away,last5games_gd_per_away,last10games_gd_per_away,last20games_gd_per_away,last5games_opp_cc_per_away,last10games_opp_cc_per_away,last20games_opp_cc_per_away
0,4,False,False,False,True,0.99,0.85,True,True,1,...,0.3,0.4,0.2,0.2,1.2,0.9,0.25,0.85,0.85,0.85
1,4,False,False,False,True,0.85,0.85,False,True,1,...,0.3,0.4,0.2,0.2,1.2,0.9,0.25,0.85,0.85,0.85
2,4,False,False,False,True,1.00,0.85,False,True,1,...,0.3,0.4,0.2,0.2,1.2,0.9,0.25,0.85,0.85,0.85
3,4,False,False,False,True,0.99,0.85,False,True,1,...,0.3,0.4,0.2,0.2,1.2,0.9,0.25,0.85,0.85,0.85
4,4,False,False,False,True,0.99,0.85,False,True,1,...,0.3,0.4,0.2,0.2,1.2,0.9,0.25,0.85,0.85,0.85


In [18]:
WorldCup_Matches_Test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 992 entries, 0 to 991
Data columns (total 42 columns):
Tournament_Weight              992 non-null int64
Friendly                       992 non-null bool
Qualification                  992 non-null bool
Final                          992 non-null bool
FIFAWorldCup                   992 non-null bool
Home_Conf_Weight               992 non-null float64
Away_Conf_Weight               992 non-null float64
Home                           992 non-null bool
Away                           992 non-null bool
Date_Weight                    992 non-null int64
Home_Team_UEFA                 992 non-null int64
Away_Team_UEFA                 992 non-null int64
last5games_w_per_home          992 non-null float64
last10games_w_per_home         992 non-null float64
last20games_w_per_home         992 non-null float64
last5games_l_per_home          992 non-null float64
last10games_l_per_home         992 non-null float64
last20games_l_per_home         992 no

In [19]:
WorldCup_Matches_Test.to_csv('WorldCup_Matches_Test.csv')

In [20]:
WorldCup_Matches_Analytics.to_csv('WorldCup_Matches_Analytics.csv')

In [21]:
Qualified_Teams_WC2018.to_csv('Qualified_Teams_WC2018.csv')

In [23]:
WorldCup2018_Analytics_Home_32.to_csv('WorldCup2018_Analytics_Home_32.csv')
WorldCup2018_Analytics_Away_32.to_csv('WorldCup2018_Analytics_Away_32.csv')